In [12]:

from fastapi import HTTPException
from typing import Any, Dict, Optional
import json
import hmac
import hashlib
import httpx

AGENT_SECRET = 'dEcartes2026'
IP = '136.111.38.85'

def json_dumps(payload: dict) -> bytes:
    return json.dumps(payload, separators=(",", ":"), sort_keys=True).encode()


def call_agent_hmac(ip, path, secret=AGENT_SECRET, json_body=None, method="POST", timeout=5):
    url = f"http://{ip}:8000{path}"
    
    if method == "DELETE":
        # Use path param (fqdn) as body for HMAC
        fqdn = path.split("/")[-1]
        body_bytes = fqdn.encode()
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers = {
            "Content-Type": "text/plain",
            "X-Signature": signature,
        }
        res = httpx.request("DELETE", url, content=body_bytes, headers=headers, timeout=timeout)

    else:
        headers = {"Content-Type": "application/json"}
        body_bytes = json_dumps(json_body) if json_body else b"{}"
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers["X-Signature"] = signature
        
        if method == "POST":
            res = httpx.post(url, content=body_bytes, headers=headers, timeout=timeout)
        elif method == "GET":
            body_bytes = b""
            signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
            headers = {"X-Signature": signature}
            res = httpx.get(url, headers=headers, timeout=timeout)

    try:
        res.raise_for_status()
    except httpx.HTTPStatusError as e:
        raise HTTPException(
            status_code=e.response.status_code,
            detail=f"Agent error: {e.response.text}"
        )
    return res


In [6]:
# Load JSON from file
with open("../json/dnsproof.org.json", "r") as f:
    payload = json.load(f)

# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/push",
    json_body=payload
)

print(response.json())

{'status': 'success'}


#### DNSSEC enabling

In [5]:
# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/sign/dnsproof.org"
)

print(response.json())

{'status': 'signed_unpublished', 'created_keys': True, 'rrsig_records': ['SOA 8 2 3600 20260209033614 20260112033614 538 @ pvNfuX3c6nRdojicRyLTB9tfC911YfF2 s3jSvpZmFIjDUQ7Ki0tGvAGciktl0u8K 6n0u5UimTM7ghn1LUbho2vCYpe7NRSz+ ffhpVGsxF10Xv/os7ZBoEd6Clu8ilDnj vQ+9MjVHgJJxilhct0O1mpXuWCo+6+8F T9h4R52/+iU=', 'NS 8 2 3600 20260209033614 20260112033614 538 @ rXTRim0iB/NJU+4El7ufW9QF/jT52yT2 wkOtNMapNGD/USrxTOmGlGr687dqhqHw KbocQRAf3C1U76aZGKR2RasagHz3dW+U S4tSwO89j4vbq9aD6fnpVaeD2ZWXxj3H xHWkdpo18tjXmqddJ02KcDtCQ778HO8I Rxtv7YZHAY8=', 'MX 8 2 3600 20260209033614 20260112033614 538 @ K5YZPKuVk8zqujRM+dd3P/o1P7GYgnoG dpbyCQ7R9UsF+N/M1bxE/gjlqic/WHdF MnWFuDMBx765Mymq3q7igGB//rwLr0LE L+o6RJqKRXwM7b+c52fyilai1TVM6iq5 mTv6AiXqby16FNONiggfVj28vDWKva/A CtkG1vapjbc=', 'TXT 8 2 3600 20260209033614 20260112033614 538 @ u7XbyIDt5YrSh861pCOdWBlSgwwcF1Xz ww6mEXX+3uVhEMkzLR2uY1tAK1By1vxz u2T0TQiQIL8DTVdDlEs1gVIy+fbg1whr hUrSjY4PgVUV/UeiqmadXtv1CpWhXMEI OrOaHPmTb7nzS+OMFYZ0zOWJ8VlG96Vk 8ey205Qz0Hs=', 'DNSKEY 8

#### DNSSEC disabling

In [4]:
# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/disable/dnsproof.org"
)

print(response.json())

{'status': 'success'}


#### DNSSEC status

In [18]:
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/status/dnsproof.org",
    method="GET"
)

print(response.json())

{'signed_at': '2026-01-12T04:05:03.560864', 'ksk_created_at': '2026-01-12T03:36:14.824968', 'zsk_created_at': '2026-01-12T03:36:14.808966', 'ds_digest': '42FBC8D9B0BC5A554C1FF2EC8A9988286B063A3B8F2DF1D6E53A78ABB3CB130B', 'key_tag': 538, 'algorithm': 8, 'days_since_last_key_creation': 0}
